In [ ]:
%pip install optuna # hyperopt, sigopt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os

import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms


DEVICE = torch.device("cpu")
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10

In [ ]:
# n_layers: 1-3 
# n_nodes: 4-128
# doprout: 0.2, 0.3, 0.4, 0.5
# optimizer: ["Adam", "RMSprop", "SGD"]
# lr: e-5, e-4, e-3, e-2, e-1

3 * 125 * 4 * 3 * 5

22500

In [ ]:

def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)


def get_mnist():
    # Load FashionMNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    return train_loader, valid_loader


def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the FashionMNIST dataset.
    train_loader, valid_loader = get_mnist()

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

[I 2023-04-15 04:04:12,559] A new study created in memory with name: no-name-cabf4d0f-4635-4116-8bc5-9cf7630070b1


100%|██████████| 26421880/26421880 [00:00<00:00, 116857064.34it/s]


Extracting /content/FashionMNIST/raw/train-images-idx3-ubyte.gz to /content/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 4457221.95it/s]


Extracting /content/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /content/FashionMNIST/raw



...

Extracting /content/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /content/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 9918363.34it/s]


Extracting /content/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /content/FashionMNIST/raw



[I 2023-04-15 04:04:25,826] Trial 0 finished with value: 0.08203125 and parameters: {'n_layers': 3, 'n_units_l0': 22, 'dropout_l0': 0.4956077789646988, 'n_units_l1': 102, 'dropout_l1': 0.2207862356370877, 'n_units_l2': 79, 'dropout_l2': 0.45346857822886527, 'optimizer': 'SGD', 'lr': 1.1733925141599031e-05}. Best is trial 0 with value: 0.08203125.
[I 2023-04-15 04:04:35,644] Trial 1 finished with value: 0.1109375 and parameters: {'n_layers': 1, 'n_units_l0': 56, 'dropout_l0': 0.33420592376376, 'optimizer': 'SGD', 'lr': 2.4765562066111482e-05}. Best is trial 1 with value: 0.1109375.
[I 2023-04-15 04:04:43,788] Trial 2 finished with value: 0.23125 and parameters: {'n_layers': 1, 'n_units_l0': 16, 'dropout_l0': 0.48952548904965953, 'optimizer': 'Adam', 'lr': 1.3181966788600786e-05}. Best is trial 2 with value: 0.23125.
[I 2023-04-15 04:04:52,346] Trial 3 finished with value: 0.76953125 and parameters: {'n_layers': 1, 'n_units_l0': 19, 'dropout_l0': 0.489089118432106, 'optimizer': 'Adam', '

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  51
  Number of complete trials:  49
Best trial:
  Value:  0.83828125
  Params: 
    n_layers: 1
    n_units_l0: 86
    dropout_l0: 0.46656652538027943
    optimizer: Adam
    lr: 0.0029311440561004105
